# 📄 生成 AT 指令文档（最终模板调整版）

In [1]:
!pip install pandas jinja2 sphinx sphinx-rtd-theme

In [2]:
import os

if not os.path.exists("docs/conf.py"):
    os.makedirs("docs/source/_static", exist_ok=True)
    with open("docs/conf.py", "w", encoding="utf-8") as f:
        f.write('''
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = []
templates_path = ['_templates']
exclude_patterns = []
html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']
def setup(app):
    app.add_css_file('custom.css')
''')
    with open("docs/source/_static/custom.css", "w", encoding="utf-8") as f:
        f.write("body { font-family: 'Microsoft YaHei', sans-serif; }")

print("✅ Sphinx 工程初始化完成")

✅ Sphinx 工程初始化完成


In [3]:
import pandas as pd
import json
from jinja2 import Environment
from collections import defaultdict

CSV_PATH = 'input.csv'
OUTPUT_DIR = 'docs/source'

TEMPLATE_STRING = """.. _cmd-{{ command_name | lower }}:

{{ command_name }} — {{ command_title }}
{{ '=' * (command_name|length + 3 + command_title|length) }}

{{ description or '' }}

执行命令
^^^^^^^^

**命令：**

::

    {{ command }}

**响应：**

::

{% for line in response.splitlines() %}    {{ line }}
{% endfor %}

参数
^^^^
{% if parameters %}
{% for p in parameters -%}
-  **{{ p.name }}**：
{% if p.valmap %}
{% for key, value in p.valmap.items() %}
   - {{ key }}：{{ value }}
{% endfor %}
{% else %}
   {{ p.desc }}
{% endif %}
{% endfor %}
{% else %}无
{% endif %}


说明
^^^^
{{ note or '无' }}

示例命令
^^^^^^^^

::

{% for line in example.splitlines() %}    {{ line }}
{% endfor %}
"""

def safe_str(val):
    return "" if pd.isna(val) else str(val).strip()

In [4]:
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()
env = Environment()
template = env.from_string(TEMPLATE_STRING)

chapters = df.groupby('章节')
chapter_commands = defaultdict(list)
chapter_names = []

for chapter, group in chapters:
    chapter = safe_str(str(chapter))
    chapter_dir = os.path.join(OUTPUT_DIR, chapter)
    os.makedirs(chapter_dir, exist_ok=True)
    chapter_names.append(chapter)

    for _, row in group.iterrows():
        cmd_name = safe_str(row['命令'])
        out_path = os.path.join(chapter_dir, f"{cmd_name}.rst")

        try:
            params = json.loads(row['参数JSON'])
        except:
            params = []

        content = template.render(
            command_name=cmd_name,
            command_title=safe_str(row['命令标题']),
            command_type=safe_str(row['命令类型']),
            command=safe_str(row['命令']),
            response=safe_str(row['响应']),
            description=safe_str(row['功能描述']),
            note=safe_str(row['备注']),
            parameters=params,
            example=safe_str(row.get('示例命令', ''))
        )

        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(content)

        chapter_commands[chapter].append(cmd_name)

print("✅ 所有 RST 文件已生成完毕")

✅ 所有 RST 文件已生成完毕


In [5]:
from jinja2 import Template

chapter_template = """{{ chapter }}
{{ '=' * chapter|length }}

.. toctree::
   :maxdepth: 1

{% for cmd in commands %}   {{ cmd }}
{% endfor %}
"""

main_template = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapters %}   {{ ch }}/index
{% endfor %}
"""

for chapter, commands in chapter_commands.items():
    ch_index = os.path.join(OUTPUT_DIR, chapter, 'index.rst')
    content = Template(chapter_template).render(chapter=chapter, commands=commands)
    with open(ch_index, 'w', encoding='utf-8') as f:
        f.write(content)

main_index = os.path.join(OUTPUT_DIR, 'index.rst')
main_content = Template(main_template).render(chapters=chapter_names)
with open(main_index, 'w', encoding='utf-8') as f:
    f.write(main_content)

print("✅ 所有章节和主目录 index.rst 已生成")

✅ 所有章节和主目录 index.rst 已生成


In [6]:
!sphinx-build -b html docs/source docs/build/html -c docs
print("✅ HTML 构建完成，可在 docs/build/html/index.html 查看")

Running Sphinx v8.2.3
loading translations [en]... done
making output directory... done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 5 source files that are out of date
updating environment: [new config] 5 added, 0 changed, 0 removed
reading sources... [100%] index;00m0m
/Users/pika/Documents/GitHub/docs-as-code-learning/learning-code/csv2pdf/docs/source/1.0/ATI.rst:4: WARNING: Title underline too short.

ATI — 获取模组厂商信息
============== [docutils]
/Users/pika/Documents/GitHub/docs-as-code-learning/learning-code/csv2pdf/docs/source/2.0/AT+CSMS.rst:4: WARNING: Title underline too short.

AT+CSMS — 选择短信服务
================ [docutils]
looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... 
copying static files... 
Writing evaluated template result to /Users/pika/Documents/GitHub/docs-as-code-learning/learning-code/csv2pdf/docs/build/html/_